In [2]:
#实验：使用DPCM编码，对随机一张灰度图片进行编码和解码，
#比较不同量化器（1-bit, 2-bit, 4-bit, 8-bit）的重建图像区别，并计算重建图像的PSNR和SSIM值。
import numpy as np
from PIL import Image
import math
import random
import matplotlib.pyplot as plt

def dpcm_encode(img_data, qbit):
    # 初始化预测值和编码数组
    predict_value = 0
    diff_codes = []
    quantized_diff_codes = []
    # 对每个像素点进行差分编码和量化
    for i in range(img_data.shape[0]):
        for j in range(img_data.shape[1]):
            diff = int(img_data[i][j]) - predict_value
            quantized_diff = round(diff / (256/(2**qbit-1))) * (256/(2**qbit-1))
            quantized_diff_codes.append(quantized_diff)
            diff_codes.append(diff)
            # 更新预测值
            predict_value += quantized_diff
    return quantized_diff_codes

def dpcm_decode(quantized_diff_codes, predict_value, qbit):
    # 初始化解码数组
    decoded_img_data = np.zeros_like(quantized_diff_codes)
    # 反量化并解码
    for i in range(len(quantized_diff_codes)):
        quantized_diff = quantized_diff_codes[i]
        diff = quantized_diff * (256/(2**qbit-1))
        predict_value += diff
        decoded_img_data[i] = predict_value
    return decoded_img_data.reshape(-1, img_width)

# 读取灰度图像
img = Image.open("touxiang.jpg").convert('L')
img_width, img_height = img.size
img_data = np.array(img)

# 随机生成预测值
predict_value = random.randint(0, 255)

# 编码和解码并比较不同量化器
for qbit in [1, 2, 4, 8]:
    # DPCM编码
    quantized_diff_codes = dpcm_encode(img_data, qbit)
    
    # 存储预测值和差分编码
    with open("dpcm_encoded_" + str(qbit) + ".bin", "wb") as f:
        # 先写入预测值
        f.write(predict_value.to_bytes(2, byteorder='big', signed=True))
        # 再写入差分编码
        for code in quantized_diff_codes:
            f.write(code.to_bytes(2, byteorder='big', signed=True))
            
    # DPCM解码
    with open("dpcm_encoded_" + str(qbit) + ".bin", "rb") as f:
        # 读取预测值
        predict_value = int.from_bytes(f.read(2), byteorder='big', signed=True)
        # 读取差分编码
        quantized_diff_codes = []
        while True:
            code_bytes = f.read(2)
            if not code_bytes:
                break
            code = int.from_bytes(code_bytes, byteorder='big', signed=True)
            quantized_diff_codes.append(code)
    decoded_img_data = dpcm_decode(quantized_diff_codes, predict_value, qbit)
    
    # 计算PSNR和SSIM
    mse = np.mean((img_data-decoded_img_data)**2)
    psnr = 20 * math.log10(255.0 / math.sqrt(mse))
    #ssim = compare_ssim(img_data, decoded_img_data, data_range=img_data.max() - img_data.min())
    print("Quantization bit: %d, PSNR: %.2f dB, SSIM: %.4f" % (qbit, psnr))


AttributeError: 'float' object has no attribute 'to_bytes'